In [10]:
#Clases
from graphviz import Digraph
import os
class Nodo:
    def __init__(self, valor, peso, tipo):
        self.valor = valor #Nombre de la imagen
        self.peso_en_bytes= peso #Peso en bytes
        self.izquierda = None
        self.derecha = None
        self.altura = 1
        self.tipo = tipo

class ArbolBinario:
    def __init__(self):
        self.raiz = None

    def altura(self, nodo):
            if nodo is None:
                return 0
            return nodo.altura

    def balance(self, nodo):
        if nodo is None:
            return 0
        return self.altura(nodo.izquierda) - self.altura(nodo.derecha)

    def _rotar_derecha(self, y):
        x = y.izquierda
        T2 = x.derecha

        x.derecha = y
        y.izquierda = T2

        y.altura = 1 + max(self.altura(y.izquierda), self.altura(y.derecha))
        x.altura = 1 + max(self.altura(x.izquierda), self.altura(x.derecha))

        return x

    def _rotar_izquierda(self, x):
        y = x.derecha
        T2 = y.izquierda

        y.izquierda = x
        x.derecha = T2

        x.altura = 1 + max(self.altura(x.izquierda), self.altura(x.derecha))
        y.altura = 1 + max(self.altura(y.izquierda), self.altura(y.derecha))

        return y


    def _balance(self, nodo, valor, peso, tipo):
        if nodo is None:
            return Nodo(valor,peso, tipo)

        if valor < nodo.valor:
            nodo.izquierda = self._balance(nodo.izquierda, valor, peso, tipo)
        elif valor > nodo.valor:
            nodo.derecha = self._balance(nodo.derecha, valor, peso, tipo)
        else:
            return nodo  # Evitar duplicados

        nodo.altura = 1 + max(self.altura(nodo.izquierda), self.altura(nodo.derecha))

        balance = self.altura(nodo.izquierda) - self.altura(nodo.derecha)

        # Casos de desequilibrio

        # Caso izquierda izquierda
        if balance > 1 and valor < nodo.izquierda.valor:
            return self._rotar_derecha(nodo)

        # Caso derecha derecha
        if balance < -1 and valor > nodo.derecha.valor:
            return self._rotar_izquierda(nodo)

        # Caso izquierda derecha
        if balance > 1 and valor > nodo.izquierda.valor:
            nodo.izquierda = self._rotar_izquierda(nodo.izquierda)
            return self._rotar_derecha(nodo)

        # Caso derecha izquierda
        if balance < -1 and valor < nodo.derecha.valor:
            nodo.derecha = self._rotar_derecha(nodo.derecha)
            return self._rotar_izquierda(nodo)

        return nodo

    def insertar(self, valor, peso, tipo):
        self.raiz = self._balance(self.raiz, valor, peso, tipo)
        #self.graficar_arbol()  # Gráfica el árbol después de cada inserción

    def eliminar_nodo(self, valor):
        if not self.buscar_nodo(valor):
            print("El nodo no existe, no se puede eliminar.")
            return
        self.raiz = self._eliminar_nodo(self.raiz, valor)
        self.graficar_arbol()  # Gráfica el árbol después de cada eliminación

    def _eliminar_nodo(self, nodo, valor):
        if nodo is None:
            return None

        # Buscar el nodo que se desea eliminar
        if valor < nodo.valor:
            nodo.izquierda = self._eliminar_nodo(nodo.izquierda, valor)
        elif valor > nodo.valor:
            nodo.derecha = self._eliminar_nodo(nodo.derecha, valor)
        else:
            # Caso: Nodo a eliminar encontrado

            # Nodo con un solo hijo o sin hijos
            if nodo.izquierda is None:
                return nodo.derecha
            elif nodo.derecha is None:
                return nodo.izquierda

            # Nodo con dos hijos
            sucesor = self._encontrar_sucesor(nodo.derecha)
            nodo.valor = sucesor.valor
            nodo.peso_en_bytes = sucesor.peso_en_bytes
            nodo.derecha = self._eliminar_nodo(nodo.derecha, sucesor.valor)

        # Actualizar altura del nodo actual
        nodo.altura = 1 + max(self.altura(nodo.izquierda), self.altura(nodo.derecha))

        # Rebalancear el árbol después de la eliminación
        return self._balance(nodo, valor, 0,"")

    def _encontrar_sucesor(self, nodo):
        while nodo.izquierda:
            nodo = nodo.izquierda
        return nodo


    def buscar_nodo(self, metrica):
        return self._buscar_nodo(self.raiz, metrica)

    def _buscar_nodo(self, nodo, metrica):
        if nodo is None or nodo.valor == metrica:
            return nodo
        elif metrica < nodo.valor:
            return self._buscar_nodo(nodo.izquierda, metrica)
        else:
            return self._buscar_nodo(nodo.derecha, metrica)


    def buscar_por_tipo_y_peso(self, tipo, peso_min, peso_max):
        resultados = []
        self._buscar_por_tipo_y_peso(self.raiz, tipo, peso_min, peso_max, resultados)
        return resultados

    def _buscar_por_tipo_y_peso(self, nodo, tipo, peso_min, peso_max, resultados):
        if nodo is None:
            return

        # Si el nodo actual cumple con los criterios, se añade a la lista de resultados
        if nodo.tipo == tipo and peso_min <= nodo.peso_en_bytes < peso_max:
            resultados.append(nodo)

        # Dado que el árbol no está organizado por tipo o peso, necesitamos recorrer todos los nodos
        self._buscar_por_tipo_y_peso(nodo.izquierda, tipo, peso_min, peso_max, resultados)
        self._buscar_por_tipo_y_peso(nodo.derecha, tipo, peso_min, peso_max, resultados)


    def graficar_arbol(self):
        if not self.raiz:
            print("El árbol está vacío.")
            return

        dot = Digraph(comment='Árbol Binario')

        def agregar_nodos_edges(nodo):
            if nodo:
                dot.node(name=str(nodo.valor), label=f'{nodo.valor}\n{str(nodo.peso_en_bytes)} bytes\n{str(nodo.tipo)}')

                if nodo.izquierda:
                    dot.edge(str(nodo.valor), str(nodo.izquierda.valor))
                    agregar_nodos_edges(nodo.izquierda)

                if nodo.derecha:
                    dot.edge(str(nodo.valor), str(nodo.derecha.valor))
                    agregar_nodos_edges(nodo.derecha)

        agregar_nodos_edges(self.raiz)

        dot.render(filename=os.path.abspath("arbol_binario"), view=False, format='svg')  # Guarda y muestra el árbol

    def _graficar_nodo(self, dot, nodo):
        if nodo:
            dot.node(str(id(nodo)), f'{nodo.valor}\n({nodo.peso_en_bytes} bytes)\nType: {nodo.tipo}')
            if nodo.izquierda:
                dot.edge(str(id(nodo)), str(id(nodo.izquierda)), label='Izquierda')
                self._graficar_nodo(dot, nodo.izquierda)
            if nodo.derecha:
                dot.edge(str(id(nodo)), str(id(nodo.derecha)), label='Derecha')
                self._graficar_nodo(dot, nodo.derecha)

    def obtener_categorias_disponibles(self):
        categorias = set()
        def recorrer(nodo):
            if nodo:
                categorias.add(nodo.tipo)
                recorrer(nodo.izquierda)
                recorrer(nodo.derecha)
        recorrer(self.raiz)
        return list(categorias)

    def mostrar_recorrido_por_niveles(self):
        def imprimir_nivel(nodo, nivel):
            if nodo is None:
                return
            if nivel == 1:
                print(nodo.valor, end=" ")
            elif nivel > 1:
                imprimir_nivel(nodo.izquierda, nivel - 1)
                imprimir_nivel(nodo.derecha, nivel - 1)

        altura = self.obtener_altura(self.raiz)
        for i in range(1, altura + 1):
            imprimir_nivel(self.raiz, i)
            print()

    def obtener_altura(self, nodo):
        if nodo is None:
            return 0
        else:
            altura_izquierda = self.obtener_altura(nodo.izquierda)
            altura_derecha = self.obtener_altura(nodo.derecha)
            return max(altura_izquierda, altura_derecha) + 1

    def obtener_nivel(self, valor, nodo, nivel=0):
      if nodo is None:
          return 0
      if nodo.valor == valor:
          return nivel
      nivel_inferior = self.obtener_nivel(valor, nodo.izquierda, nivel + 1)
      if nivel_inferior != 0:
          return nivel_inferior
      nivel_inferior = self.obtener_nivel(valor, nodo.derecha, nivel + 1)
      return nivel_inferior

    def encontrar_relaciones(self, valor):
        def buscar_y_relacionar(nodo, valor_busqueda, ancestros=[]):
            if nodo is None:
                return None, None, None
            if nodo.valor == valor_busqueda:
                padre = ancestros[-1] if len(ancestros) >= 1 else None
                abuelo = ancestros[-2] if len(ancestros) >= 2 else None
                tio = None
                if abuelo:
                    if abuelo.izquierda is not None and abuelo.izquierda != padre:
                        tio = abuelo.izquierda
                    elif abuelo.derecha is not None and abuelo.derecha != padre:
                        tio = abuelo.derecha
                return padre, abuelo, tio
            else:
                # Intenta buscar primero en la izquierda, y si no se encuentra, en la derecha
                result = buscar_y_relacionar(nodo.izquierda, valor_busqueda, ancestros + [nodo])
                if result == (None, None, None):  # Si no se encuentra en la izquierda, busca en la derecha
                    return buscar_y_relacionar(nodo.derecha, valor_busqueda, ancestros + [nodo])
                return result

        return buscar_y_relacionar(self.raiz, valor)



In [11]:
#ABRIR ZIP FILE


import zipfile
import os

# Nombre del archivo ZIP cargado
zip_filename = os.path.abspath("data.zip")

# Ruta de destino para la extracción
extract_path = os.path.abspath("extracted")

# Crea el directorio de extracción si no existe
os.makedirs(extract_path, exist_ok=True)

# Descomprime el archivo ZIP
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [12]:
#LECTURA DE LAS IMAGENES

from PIL import Image

def leer_carpeta_con_imagenes(carpeta_raiz):
    for carpeta_actual, subcarpetas, archivos in os.walk(carpeta_raiz):
        for archivo in archivos:
            if archivo.lower().endswith((".bmp", ".jpg", ".jpeg", ".png")):
                ruta_completa = os.path.join(carpeta_actual, archivo)
                procesar_imagen(ruta_completa)

def procesar_imagen(ruta_imagen):
    imagen = Image.open(ruta_imagen)

    print(obtener_nombre_sin_extension(ruta_imagen))

    peso_en_bytes = obtener_peso_en_bytes(ruta_imagen)
    print("Peso de la imagen en bytes:", peso_en_bytes)

    tipo = os.path.basename(os.path.dirname(ruta_imagen))
    print("Carpeta de origen:", tipo)

    arbol.insertar(obtener_nombre_sin_extension(ruta_imagen), peso_en_bytes, tipo)


    imagen.close()

def obtener_peso_en_bytes(ruta_imagen):
    return os.path.getsize(ruta_imagen)


def obtener_nombre_sin_extension(ruta_archivo):
    nombre_base = os.path.basename(ruta_archivo)
    return os.path.splitext(nombre_base)[0]


# Ruta de la carpeta principal que contiene las subcarpetas con imágenes
carpeta_principal = os.path.abspath("extracted")

arbol= ArbolBinario()

# Llama a la función para leer la carpeta con imágenes
leer_carpeta_con_imagenes(carpeta_principal)
#arbol.eliminar_nodo("0005")
arbol.graficar_arbol()

bike_001
Peso de la imagen en bytes: 921654
Carpeta de origen: bike
bike_002
Peso de la imagen en bytes: 921654
Carpeta de origen: bike
bike_003
Peso de la imagen en bytes: 921654
Carpeta de origen: bike
bike_004
Peso de la imagen en bytes: 921654
Carpeta de origen: bike
bike_005
Peso de la imagen en bytes: 921654
Carpeta de origen: bike
bike_006
Peso de la imagen en bytes: 921654
Carpeta de origen: bike
bike_007
Peso de la imagen en bytes: 921654
Carpeta de origen: bike
bike_008
Peso de la imagen en bytes: 921654
Carpeta de origen: bike
bike_009
Peso de la imagen en bytes: 921654
Carpeta de origen: bike
bike_010
Peso de la imagen en bytes: 921654
Carpeta de origen: bike
bike_011
Peso de la imagen en bytes: 921654
Carpeta de origen: bike
bike_012
Peso de la imagen en bytes: 921654
Carpeta de origen: bike
bike_013
Peso de la imagen en bytes: 921654
Carpeta de origen: bike
bike_014
Peso de la imagen en bytes: 921654
Carpeta de origen: bike
bike_015
Peso de la imagen en bytes: 921654
Carp

In [13]:

#MENÚ
while True:
    print("\nMenú Principal del Árbol Binario:")
    print("1. Insertar Nodo")
    print("2. Eliminar Nodo")
    print("3. Buscar un Nodo utilizando la métrica")
    print("4. Buscar Nodos bajo criterio")
    print("5. Mostrar recorrido por niveles del árbol")
    print("6. Operaciones de Nodo Específico")
    print("7. Salir")

    choice = input("Ingrese su elección (1-7): ").strip()

    if not choice.isdigit() or not 1 <= int(choice) <= 7:
        print("Por favor, ingrese un número válido entre 1 y 7.")
        continue

    if choice == "1":
        valor = input("Ingrese el nombre de la imagen a insertar: ").strip()
        peso = input("Ingrese el peso en bytes de la imagen: ").strip()

        if not peso.isdigit() or int(peso) < 0:
            print("El peso debe ser un número entero positivo. Inténtelo de nuevo.")
            continue

        print("Categorías disponibles: ", ", ".join(arbol.obtener_categorias_disponibles()))
        tipo = input("Ingrese el tipo/categoría de la imagen: ").strip()

        if tipo not in arbol.obtener_categorias_disponibles():
            print("Advertencia: Ha seleccionado una categoría que no está en la lista de categorías disponibles.")
            confirmar = input("¿Desea continuar con esta nueva categoría? (sí/no): ").strip().lower()
            if confirmar != 'sí' and confirmar != 'si':
                continue

        arbol.insertar(valor, int(peso), tipo)
        arbol.graficar_arbol()
        print("Nodo insertado exitosamente y árbol actualizado.")


    elif choice == "2":
        valor = input("Ingrese el nombre de la imagen a eliminar: ").strip()
        if not arbol.buscar_nodo(valor):
            print("El nodo con el valor especificado no existe.")
        else:
            arbol.eliminar_nodo(valor)
            print("Nodo eliminado exitosamente y árbol actualizado.")

    elif choice == "3":
        metrica = input("Ingrese el nombre de la imagen a buscar: ").strip()
        if arbol.buscar_nodo(metrica):
            print(f"Nodo encontrado: {metrica}")
            nodo=arbol.buscar_nodo(metrica)
            ###
            nivel = arbol.obtener_nivel(metrica, arbol.raiz)
            balance = arbol.balance(nodo)
            relaciones= arbol.encontrar_relaciones(metrica)
            if relaciones:
              padre, abuelo, tio = relaciones


            if tio!=None:
                valortio=tio.valor
            else:
                valortio=None
            if abuelo!=None:
                valorabuelo=abuelo.valor
            else:
                valorabuelo=None
            if padre!=None:
                valorpadre=padre.valor
            else:
                valorpadre=None

            print(f"Detalles del nodo:\n- Nivel: {nivel}\n- Factor de balance: {balance}\n- Padre: {valorpadre}\n- Abuelo: {valorabuelo}\n- Tío: {valortio}")
            ###

        else:
            print("Nodo no encontrado.")

    elif choice == "4":
        print("Categorías disponibles: ", ", ".join(arbol.obtener_categorias_disponibles()))
        tipo = input("Ingrese el tipo/categoría para la búsqueda: ").strip()

        if tipo not in arbol.obtener_categorias_disponibles():
            print("Tipo no válido. Inténtelo de nuevo con una categoría válida.")
            continue

        peso_min = input("Ingrese el peso mínimo en bytes: ").strip()
        peso_max = input("Ingrese el peso máximo en bytes: ").strip()

        if not peso_min.isdigit() or not peso_max.isdigit() or int(peso_min) > int(peso_max):
            print("Los pesos deben ser números enteros, y el peso mínimo no puede ser mayor que el peso máximo.")
            continue

        nodos = arbol.buscar_por_tipo_y_peso(tipo, int(peso_min), int(peso_max))
        if nodos:
            print("Nodos encontrados:")
            encontrados = []
            for i, nodo in enumerate(nodos):
              print(f"{i+1}. {nodo.valor}")

              encontrados.append(nodo)
            while True:
              seleccion = input("Seleccione el número del nodo para realizar las operaciones (o '0' para volver atrás): ").strip()
              if seleccion == '0':
                  break
              elif not seleccion.isdigit() or int(seleccion) < 1 or int(seleccion) > len(nodos):
                  print("Selección no válida. Introduzca un número válido o '0' para volver atrás.")
              else:
                nivel = arbol.obtener_nivel(encontrados[int(seleccion)-1].valor, arbol.raiz)
                balance = arbol.balance(encontrados[int(seleccion)-1])
                relaciones= arbol.encontrar_relaciones(encontrados[int(seleccion)-1].valor)
                if relaciones:
                  padre, abuelo, tio = relaciones

                if tio!=None:
                    valortio=tio.valor
                else:
                    valortio=None
                if abuelo!=None:
                    valorabuelo=abuelo.valor
                else:
                    valorabuelo=None
                if padre!=None:
                    valorpadre=padre.valor
                else:
                    valorpadre=None

                print(f"Detalles del nodo:\n- Nivel: {nivel}\n- Factor de balance: {balance}\n- Padre: {valorpadre}\n- Abuelo: {valorabuelo}\n- Tío: {valortio}")


        else:
            print("No se encontraron nodos que cumplan con los criterios.")

    elif choice == "5":
        print("Mostrando el recorrido por niveles del árbol:")
        arbol.mostrar_recorrido_por_niveles()

    elif choice == "6":
        valor = input("Ingrese el nombre del nodo para obtener sus detalles: ").strip()
        nodo = arbol.buscar_nodo(valor)
        if nodo is None:
            print("Nodo no encontrado.")
        else:
            nivel = arbol.obtener_nivel(valor, arbol.raiz)
            balance = arbol.balance(nodo)
            relaciones= arbol.encontrar_relaciones(valor)
            if relaciones:
              padre, abuelo, tio = relaciones
            print(f"Detalles del nodo:\n- Nivel: {nivel}\n- Factor de balance: {balance}\n- Padre: {padre.valor}\n- Abuelo: {abuelo.valor}\n- Tío: {tio.valor}")

    elif choice == "7":
        print("Saliendo del programa...")
        break


Menú Principal del Árbol Binario:
1. Insertar Nodo
2. Eliminar Nodo
3. Buscar un Nodo utilizando la métrica
4. Buscar Nodos bajo criterio
5. Mostrar recorrido por niveles del árbol
6. Operaciones de Nodo Específico
7. Salir
Saliendo del programa...
